* 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파(backpropagation)
* 이 알고리즘에서 파라미터(모델 가중치)는 주어진 파라미터에 대한 손실 함수의 기울기에 따라 조정
* 이러한 기울기를 계산하기 위해 파이토치에는 torch.autograd라는 미분 엔진이 내장
* 이 엔진은 모든 계산 그래프에 대한 기울기 자동 계산을 지원

In [10]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

### 텐서, 함수 및 계산 그래프
* 이 네트워크에서 w와 b는 최적화해야 하는 파라미터
* 따라서 이러한 변수에 대한 손실 함수의 기울기를 계산할 수 있어야 합니다. 이를 위해 해당 텐서의 requires_grad 속성을 설정
* 역전파 함수에 대한 참조는 텐서의 grad_fn 속성에 저장

In [11]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x000001736A06F760>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001736A06E770>


In [4]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

y = x + 2
print(y)
print(y.grad_fn)

tensor([3., 4., 5.], grad_fn=<AddBackward0>)


* 덧셈 연산에 대한 기록을 `grad_fn'의 속성을 통해 확인

In [5]:
z = y * 3
print(z)

out = z.mean()
print(out)

tensor([ 9., 12., 15.], grad_fn=<MulBackward0>)
tensor(12., grad_fn=<MeanBackward0>)


* 곱셈 연산에 대한 기록 확인

### 그래디언트 계산하기
* 신경망에서 파라미터의 가중치를 최적화하려면 파라미터에 대한 손실 함수의 미분을 계산
* 이러한 도함수를 계산하기 위해 loss.backward()를 호출한 다음 w.grad 및 b.grad에서 값을 검색

In [12]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0959, 0.0140, 0.3074],
        [0.0959, 0.0140, 0.3074],
        [0.0959, 0.0140, 0.3074],
        [0.0959, 0.0140, 0.3074],
        [0.0959, 0.0140, 0.3074]])
tensor([0.0959, 0.0140, 0.3074])


* 이 노드에는 requires_grad 속성이 True로 설정되어 있습니다. 

### 그래디언트 추적 비활성화하기
* 기본적으로 requires_grad가 True인 모든 텐서는 계산 이력을 추적하고 그래디언트 계산을 지원
* 모델을 학습시킨 후 일부 입력 데이터에만 적용하려는 경우, 즉 네트워크를 통해 순방향 계산만 수행하려는 경우와 같이 그렇게 할 필요가 없는 경우도 있습니다. 
* 계산 코드를 torch.no_grad() 블록으로 둘러싸서 계산 추적을 중지할 수 있습니다.

In [13]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [14]:
# 동일한 결과를 얻는 또 다른 방법은 텐서에서 detach() 메서드를 사용하는 것입니다.
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


#### Forward 전달에서 autograd는 두 가지 작업을 동시에 수행합니다.
* 요청된 연산을 실행하여 결과 텐서를 계산합니다.
* DAG에서 연산의 그래디언트 함수를 유지합니다.

In [15]:
# 텐서를 생성하고 requires_grad를 True로 설정합니다.
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 제곱의 합을 연산
y = x * x
z = y.sum()

# 그라데이션 계산을 위한 backward 전달
z.backward()

# 이제 x.grad는 x에 대한 z의 기울기를 유지합니다.
print(x.grad)  # Output: tensor([2., 4., 6.])

tensor([2., 4., 6.])


#### backward 전달에서는 DAG 루트에서 .backward()가 호출될 때 시작됩니다.
1. autograd는 각 .grad_fn에서 그래디언트를 계산
2. 각 텐서의 .grad 속성에 누적하고,
3. 체인 규칙을 사용하여 잎(leaf) 텐서까지 전파합니다.


In [16]:
# 텐서를 생성하고 requires_grad를 True로 설정합니다.
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 제곱의 합을 연산
t = x * x
y = t * t
z = y.sum()

# 그라데이션 계산을 위한 backward 전달
z.backward()

# 이제 x.grad는 x에 대한 z의 기울기를 유지합니다.
print(x.grad)  # Output: tensor([2., 4., 6.])

tensor([  4.,  32., 108.])


### 텐서 그래디언트와 자코비안 곱
* 대부분의 경우 스칼라 손실 함수가 있고 일부 매개변수에 대한 기울기를 계산해야 합니다. 
* 그러나 출력 함수가 임의의 텐서인 경우도 있습니다.
* 이 경우 PyTorch를 사용하면 실제 그라데이션이 아닌 소위 자코비안 곱을 계산할 수 있습니다.
* 이를 위해 다음과 같이 v 를 인수로 사용하여 backward를 호출하여 얻을 수 있습니다.
* v의 크기는 곱을 계산하려는 원래 텐서의 크기와 같아야 합니다.

In [17]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
